In [ ]:
import numpy as np
import hyperspy.api as hys
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
import tifffile
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
ini_channel = 280.0
fin_channel = 315.0
dispersion = 0.025
e_range = np.arange(ini_channel, fin_channel, dispersion)
print(e_range.shape)

In [ ]:
spectra_adr = tkf.askopenfilenames()
print(*spectra_adr, sep="\n")

In [ ]:
spectra = tifffile.imread(spectra_adr)
print(spectra.shape)
spectra = spectra.squeeze()
print(spectra.shape)

In [ ]:
spectra = []
for adr in spectra_adr:
    tmp = hys.load(adr, signal_type="EELS")
    tmp = tmp.isig[ini_channel:fin_channel]
    data = tmp.data.clip(min=0.0)
    tmp_spectrum = np.mean(data, axis=(0, 1))
    spectra.append(tmp_spectrum)
    
spectra = np.asarray(spectra)
print(spectra.shape)

In [ ]:
spectra = []
for adr in spectra_adr:
    tmp = hys.load(adr, signal_type="EELS")
    tmp = tmp.isig[ini_channel:fin_channel]
    spectra.append(tmp.data)
    
spectra = np.asarray(spectra)
print(spectra.shape)

In [ ]:
num_s = len(spectra)
print(num_s)

In [ ]:
for i, spec in enumerate(spectra):
    spectra[i] = spec - np.min(spec)

In [ ]:
pi_peak = find_nearest(e_range, 285.0)
print(pi_peak)

ini_ind_align = find_nearest(e_range, 282.0)
fin_ind_align = find_nearest(e_range, 287.0)
pi_peak_ind = []
for i in range(num_s):
    pi_peak_ind.append(ini_ind_align+np.argmax(spectra[i][ini_ind_align:fin_ind_align]))
    
pi_peak_ind = np.asarray(pi_peak_ind)
print(pi_peak_ind)

In [ ]:
fig, ax = plt.subplots(num_s, 1, figsize=(5, num_s*3))
for i in range(num_s):
    ax[i].plot(spectra[i], 'k-')
    ax[i].fill_between([pi_peak_ind[i], pi_peak_ind[i]+1], np.max(spectra[i]), 
                alpha=0.5, color="red")
    ax[i].grid()
fig.tight_layout()
plt.show()

In [ ]:
pi_ind_diff = pi_peak_ind - pi_peak
print(pi_ind_diff)

In [ ]:
e_ranges = []
for i in range(num_s):
    tmp = np.arange(ini_channel-pi_ind_diff[i]*dispersion, fin_channel-pi_ind_diff[i]*dispersion, dispersion)
    e_ranges.append(tmp)
    print(tmp[0], tmp[-1])
    
e_ranges = np.asarray(e_ranges)

In [ ]:
pi_onset = 280.0
pi_final = 288.0
sigma_final = 310.0
print(sigma_final)

In [ ]:
pi_ranges = []
pi_ranges_spectra = []
for i in range(num_s):
    left_ind = (pi_onset - e_ranges[i][0]) / dispersion
    right_ind = (pi_final - e_ranges[i][0]) / dispersion

    pi_range = e_ranges[i][int(left_ind):int(right_ind)]
    pi_range_spectra = spectra[i][int(left_ind):int(right_ind)]
    print(pi_range.shape)
    print(pi_range[0], pi_range[-1])
    print(pi_range_spectra.shape)
    
    pi_ranges.append(pi_range)
    pi_ranges_spectra.append(pi_range_spectra)

In [ ]:
pi_sigma_ranges = []
pi_sigma_ranges_spectra = []
for i in range(num_s):
    left_ind = (pi_onset - e_ranges[i][0]) / dispersion
    right_ind = (sigma_final - e_ranges[i][0]) / dispersion

    pi_sigma_range = e_ranges[i][int(left_ind):int(right_ind)]
    pi_sigma_range_spectra = spectra[i][int(left_ind):int(right_ind)]
    print(pi_sigma_range.shape)
    print(pi_sigma_range[0], pi_sigma_range[-1])
    print(pi_sigma_range_spectra.shape)
    
    pi_sigma_ranges.append(pi_sigma_range)
    pi_sigma_ranges_spectra.append(pi_sigma_range_spectra)

In [ ]:
fig, ax = plt.subplots(num_s, 1, figsize=(5, num_s*3))
for i in range(num_s):
    ax[i].plot(e_ranges[i], spectra[i], 'k-')
    Epi = np.trapz(pi_ranges_spectra[i], pi_ranges[i])
    Episigma = np.trapz(pi_sigma_ranges_spectra[i], pi_sigma_ranges[i])
    #print(Epi, Esigma)
    sp23_ratio = Epi / Episigma
    print(sp23_ratio)
    ax[i].set_title("label %d, sp2/sp3 ratio=%f"%(i, sp23_ratio))
    ax[i].fill_between([pi_ranges[i][0], pi_ranges[i][-1]], np.max(spectra[i]), 
                alpha=0.5, color="red")
    ax[i].fill_between([pi_sigma_ranges[i][0], pi_sigma_ranges[i][-1]], np.max(spectra[i]), 
                alpha=0.5, color="blue")
    ax[i].grid()
#fig.legend(fontsize="x-large")
fig.tight_layout()
plt.show()